<a href="https://colab.research.google.com/github/sanjal02/Early-Disease-Prediction/blob/main/File_conversion_and_preliminary_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xport

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 35.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas: filename=pandas-1.3.5-cp311-cp311-linux_x86_64.whl size=37464137 sha256=7923237f2959c8af0c337a7250fd5fa17b134bb836b948d22a82a23f1952b8de
  Stored in directory: /root/.cache/pip/wheels/8b/e7/6d/d4c288f419ab8fa07c1db6f606a2ae18ecf3dc2839d79a1c07
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.3.5 which is incompatible.
bigframes 1.35.0 requires pandas>=1.5.3, but you have pandas 1.3.5 wh

In [ ]:
# Required Imports
import xport, csv
import os
import pandas as pd

In [ ]:
# Directory path
data_dir = '/content/VID_L.xpt'

In [ ]:
import xport, csv
import os
import pandas as pd

# Function to convert an .XPT file to .csv format
def xpt_to_csv(filepath, savedir):
    with open(filepath, 'rb') as f:
        # Overriding the default encoding
        xport.v56.TEXT_METADATA_ENCODING = 'latin-1'  # or 'utf-8' if latin-1 doesn't work
        df = xport.to_dataframe(f)
    print(df.columns.values)
    name = os.path.basename(filepath).split('.')[0] # Get the filename without the extension
    savepath = os.path.join(savedir, name + '.csv') # Construct the save path using os.path.join
    df.to_csv(savepath, index=False) # Save the DataFrame to CSV, avoid saving index

    return True

In [ ]:
xpt_to_csv(data_dir, '.') # Call the xpt_to_csv function with the file path and save directory

['SEQN' 'WTPH2YR' 'LBXVIDMS' 'LBDVIDLC' 'LBXVD2MS' 'LBDVD2LC' 'LBXVD3MS'
 'LBDVD3LC' 'LBXVE3MS' 'LBDVE3LC']


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

def merge_files_on_seqn(file_list, output_file):
    """Merges multiple CSV files on the SEQN column, keeping all columns."""
    merged_df = None  # Initialize an empty DataFrame

    for filename in file_list:
        try:
            df = pd.read_csv(filename)

            # Ensure SEQN column exists
            if 'SEQN' not in df.columns:
                print(f"Warning: 'SEQN' column not found in {filename}. Skipping file.")
                continue

            # Merge on SEQN, using outer join to keep all columns
            if merged_df is None:
                merged_df = df  # First file sets the base dataframe
            else:
                merged_df = pd.merge(merged_df, df, on='SEQN', how='outer')

        except FileNotFoundError:
            print(f"Error: File not found: {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    # Save the merged dataframe
    if merged_df is not None:
        merged_df.to_csv(output_file, index=False)
        print(f"Files merged successfully into {output_file}")
    else:
        print("No valid files found for merging.")

# Example usage:
files_to_merge = [
    '/content/Alcohol_Use.csv', '/content/Alpha-1_Glycoprotein.csv', '/content/Blood Pressure.csv',
    '/content/Body Measures.csv', '/content/Complete_Blood_Count.csv', '/content/Current_Health_Stats.csv',
    '/content/Demographics.csv', '/content/Diabetes.csv', '/content/Diet_Behavior&Nutrition.csv',
    '/content/Fast_Questionnaire.csv', '/content/Feritin.csv', '/content/Folate(RBC).csv',
    '/content/Glycohemoglobin.csv', '/content/Hep_A.csv', '/content/Hep_B_Surface_Antibody.csv',
    '/content/High_Sensitivity_C_Protein.csv', '/content/Immunization.csv', '/content/Insulin.csv',
    '/content/Lead, Cadmium, Total Mercury, Selenium, & Manganese.csv', '/content/Liver Ultrasound.csv',
    '/content/Medical_Conditions.csv', '/content/Mercury_ Inorganic, Ethyl, and Methyl.csv',
    '/content/Nutrient_Intake_day_1.csv', '/content/Nutrient_Intake_day_2.csv', '/content/Occupations.csv',
    '/content/Physical_Activity.csv', '/content/Plasma_Fasting_Glucose.csv', '/content/Prescription_Meds.csv',
    '/content/Serum_Folate.csv', '/content/Sex_Steroid_Hormone.csv', '/content/Sleep_Disorder.csv',
    '/content/Smoking.csv', '/content/Total_Cholestrol.csv', '/content/Vitamin_D.csv', '/content/Weight_History.csv'
]
output_filename = 'merged_file.csv'
merge_files_on_seqn(files_to_merge, output_filename)

<ipython-input-92-122d451c490c>:21: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, df, on='SEQN', how='outer')
<ipython-input-92-122d451c490c>:21: FutureWarning: Passing 'suffixes' which cause duplicate columns {'WTPH2YR_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, df, on='SEQN', how='outer')


Files merged successfully into merged_file.csv


In [ ]:
import pandas as pd
from google.colab import files

# Download the merged_file.csv
files.download('merged_file_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

def identify_missing_columns(filepath, threshold=0.7):
    """
    Identifies columns in a CSV file with more than the specified threshold of missing data.

    Args:
        filepath: Path to the CSV file.
        threshold: The proportion of missing data (0 to 1) above which a column is flagged.

    Returns:
        A list of column names with more than the threshold proportion of missing values.
    """

    try:
        df = pd.read_csv(filepath)
        missing_percentage = df.isnull().sum() / len(df)
        columns_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()
        return columns_to_drop

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
filepath = '/content/merged_file.csv'  # Replace with the actual file path
missing_cols = identify_missing_columns(filepath)

if missing_cols:
    print("Columns with more than 70% missing data:")
    for col in missing_cols:
        print(col)
else:
    print("No columns with more than 70% missing data found.")


Columns with more than 70% missing data:
ALQ270
ALQ280
ALQ170
WTPH2YR_x
LBXAGP
BMIWT
BMXRECUM
BMIRECUM
BMXHEAD
BMIHEAD
BMIHT
BMDBMIC
BMILEG
BMIARML
BMIARMC
BMIWAIST
BMIHIP
RIDAGEMN
RIDEXAGM
DMDYRUSR
RIDEXPRG
DMDHSEDZ
DID040
DIQ050
DID060
DIQ060U
DIQ070
DBQ010
DBD030
DBD041
DBD050
DBD055
DBD061
DBQ073A
DBQ073B
DBQ073C
DBQ073D
DBQ073E
DBQ073U
DBQ301
DBQ330
DBQ360
DBQ370
DBD381
DBQ390
DBQ400
DBD411
DBQ421
DBQ424
PHACOFHR
PHACOFMN
PHAALCHR
PHAALCMN
PHAGUMHR
PHAGUMMN
PHAANTHR
PHAANTMN
PHASUPHR
PHASUPMN
WTPH2YR_x.1
LBXFER
LBDFERSI
IMQ060
IMQ070
IMQ090
IMQ100
LBXIN
LBDINSI
LBDINLC
LUARXNC
LUARXND
LUARXIN
MCQ035
MCQ040
MCQ050
MCQ149
MCQ195
MCQ170M
MCQ170L
MCQ500
MCQ510A
MCQ510B
MCQ510C
MCQ510D
MCQ510E
MCQ510F
MCQ230A
MCQ230B
MCQ230C
MCQ230D
DBD100
DR1SKY
DRQSDT1
DRQSDT2
DRQSDT3
DRQSDT4
DRQSDT5
DRQSDT6
DRQSDT7
DRQSDT8
DRQSDT9
DRQSDT10
DRQSDT11
DRQSDT12
DRQSDT91
DRD350A
DRD350AQ
DRD350B
DRD350BQ
DRD350C
DRD350CQ
DRD350D
DRD350DQ
DRD350E
DRD350EQ
DRD350F
DRD350FQ
DRD350G
DRD350GQ
DRD350H
DRD350HQ

<ipython-input-94-4df163e38fbb>:30: DtypeWarning: Columns (13,197,509,512) have mixed types.Specify dtype option on import or set low_memory=False.
  missing_cols = identify_missing_columns(filepath)


In [ ]:
# prompt: delete all the above identified columns

import pandas as pd

def delete_columns(filepath, columns_to_delete):
    """Deletes specified columns from a CSV file and saves the result."""
    try:
        df = pd.read_csv(filepath)

        # Check if all columns to delete exist in the DataFrame
        for col in columns_to_delete:
            if col not in df.columns:
                print(f"Warning: Column '{col}' not found in the DataFrame. Skipping.")

        df = df.drop(columns=[col for col in columns_to_delete if col in df.columns])
        df.to_csv(filepath.replace('.csv', '_clean.csv'), index=False)  # Save with _clean suffix
        print(f"Columns deleted and saved to {filepath.replace('.csv', '_clean.csv')}")
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage (replace with the actual file path and columns to delete):
filepath = '/content/merged_file.csv'
# Assuming 'missing_cols' is the list of columns identified in the previous step
delete_columns(filepath, missing_cols)


<ipython-input-95-99b9cd2d0c0d>:26: DtypeWarning: Columns (13,197,509,512) have mixed types.Specify dtype option on import or set low_memory=False.
  delete_columns(filepath, missing_cols)


Columns deleted and saved to /content/merged_file_clean.csv
